In [73]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
from sklearn.metrics import plot_precision_recall_curve
import numpy as np
from sklearn.model_selection import GridSearchCV


[nltk_data] Downloading package punkt to /home/motherlode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/motherlode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [74]:
df = pd.read_csv("data/Corona_NLP_train.csv", encoding='latin-1')
df["Sentiment"].replace(to_replace = {'Extremely Positive' : 1, 'Positive' : 1, 'Neutral' : 1, 
                                   'Negative' : 0, 'Extremely Negative' : 0}, inplace=True)
df_test = pd.read_csv("data/Corona_NLP_test.csv", encoding='latin-1')
df_test["Sentiment"].replace(to_replace = {'Extremely Positive' : 1, 'Positive' : 1, 'Neutral' : 1, 
                                   'Negative' : 0, 'Extremely Negative' : 0}, inplace=True)
print(df_test)

      UserName  ScreenName             Location     TweetAt  \
0            1       44953                  NYC  02-03-2020   
1            2       44954          Seattle, WA  02-03-2020   
2            3       44955                  NaN  02-03-2020   
3            4       44956          Chicagoland  02-03-2020   
4            5       44957  Melbourne, Victoria  03-03-2020   
...        ...         ...                  ...         ...   
3793      3794       48746            Israel ??  16-03-2020   
3794      3795       48747       Farmington, NM  16-03-2020   
3795      3796       48748        Haverford, PA  16-03-2020   
3796      3797       48749                  NaN  16-03-2020   
3797      3798       48750  Arlington, Virginia  16-03-2020   

                                          OriginalTweet  Sentiment  
0     TRENDING: New Yorkers encounter empty supermar...          0  
1     When I couldn't find hand sanitizer at Fred Me...          1  
2     Find out how you can protect y

In [75]:
df.shape
#df_test["Sentiment"].value_counts()
#df - тренирвка. dt_test - test

(41157, 6)

In [76]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,1
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,1
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,1
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0


In [77]:
df["Sentiment"].value_counts()

1    25759
0    15398
Name: Sentiment, dtype: int64

In [78]:
snowball = SnowballStemmer(language="english")
english_stop_words = stopwords.words("english")

def tokenize_sentence(sentence: str, remove_stop_words:bool = True):
    tokens = tokens = word_tokenize(sentence, language="english")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in english_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [84]:
t = df["OriginalTweet"][9:10]
print(t)

9    For corona prevention,we should stop to buy th...
Name: OriginalTweet, dtype: object


In [85]:
tok = [tokenize_sentence(x) for x in t]
print(tok)

[['for', 'corona', 'prevent', 'stop', 'buy', 'thing', 'cash', 'use', 'onlin', 'payment', 'method', 'corona', 'spread', 'note', 'also', 'prefer', 'onlin', 'shop', 'home', 'it', "'s", 'time', 'fight', 'covid', '19', 'govindia', 'indiafightscorona']]


In [87]:
tfidf = TfidfVectorizer(min_df=1, stop_words='english')
x_demo = tfidf.fit_transform(t)
idf = tfidf.idf_
d = dict(zip(tfidf.get_feature_names(), idf))
print(d)

{'19': 1.0, 'buy': 1.0, 'cash': 1.0, 'corona': 1.0, 'covid': 1.0, 'fight': 1.0, 'govindia': 1.0, 'home': 1.0, 'indiafightscorona': 1.0, 'methods': 1.0, 'notes': 1.0, 'online': 1.0, 'payment': 1.0, 'prefer': 1.0, 'prevention': 1.0, 'shopping': 1.0, 'spread': 1.0, 'stop': 1.0, 'things': 1.0, 'time': 1.0, 'use': 1.0}


In [9]:
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x))),
    ("model", LogisticRegression(C = 10.0, random_state = 0, max_iter = 500))
]    
)

/home/motherlode/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function <lambda> at 0x7f2ed32dfd90>)

In [44]:
model_pipeline.fit(df["OriginalTweet"], df["Sentiment"])

NameError: name 'model_pipeline' is not defined

In [11]:
precision_score(y_true=df_test["Sentiment"], y_pred=model_pipeline.predict(df_test["OriginalTweet"]))

0.8266267123287672

In [12]:
recall_score(y_true=df_test["Sentiment"], y_pred=model_pipeline.predict(df_test["OriginalTweet"]))

0.8919168591224018

In [13]:
prec, rec, thresholds = y_true=df_test["Sentiment"], probas_pred=model_pipeline.predict_proba(df_test["OriginalTweet"])[:, 1])

SyntaxError: unmatched ')' (1949966827.py, line 1)

In [ ]:
plot_precision_recall_curve(estimator=model_pipeline, X=df_test["OriginalTweet"], y=df_test["Sentiment"])

In [ ]:
np.where(prec > 0.95)
#model_pipeline.predict(["good."])

In [ ]:
thresholds[2756]

In [ ]:
precision_score(y_true=df_test["Sentiment"], y_pred=model_pipeline.predict_proba(df_test["OriginalTweet"])[:, 1] > thresholds[2756])

In [ ]:
recall_score(y_true=df_test["Sentiment"], y_pred=model_pipeline.predict_proba(df_test["OriginalTweet"])[:, 1] > thresholds[2756])

In [ ]:
w = ["feel good", "is ill", "cagh", "weakness in the legs", "exelent", "normal state of health"]

In [ ]:
ans = model_pipeline.predict_proba(w)
ans = ans[:, 1]

In [ ]:
pd.crosstab(w , ans).plot(kind = 'bar')

table=pd.crosstab(data.marital,data.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)


In [ ]:
%matplotlib inline
table=pd.crosstab(w,ans)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import csv
dpi = 80
fig = plt.figure(dpi = dpi, figsize = (512 / dpi, 384 / dpi) )
mpl.rcParams.update({'font.size': 9})
ax = plt.axes()
ax.xaxis.grid(True, zorder = 1)
xs = range(len(w))

plt.barh([x + 0.3 for x in xs], [ 1 - d for d in ans],
         height = 0.2, color = 'red', alpha = 0.7, label = 'bad',
         zorder = 2)
plt.barh([x + 0.05 for x in xs], ans,
         height = 0.2, color = 'blue', alpha = 0.7, label = 'good',
         zorder = 2)
plt.yticks(xs, w, rotation = 10)

plt.legend(loc='upper right')
fig.savefig('barshoris.png')